In [2]:
# checked with http://pep8online.com/checkresult
# cmake --preset vtk-9.1.0-vs2019-x64
#
import subprocess
import re
import os

DUMPBIN = (os.environ['VS2019INSTALLDIR'] +
           '\\VC\\Tools\\MSVC\\14.29.30133\\bin\\HostX86\\x86\\dumpbin.exe')
MVMF6_EXE = '..\\_vtk-9.1.0-vs2019-x64\\ModelViewer\\Release\\mvmf6.exe'
MV_DLL = '..\\_vtk-9.1.0-vs2019-x64\\ModelViewer\\Release\\mv.dll'
VTK_BIN = 'C:\\VTK-9.1.0-vs2019-x64\\bin'
DLL_RE = re.compile('[ ].*vtk.*-9.1.dll')


def depends(dll_dict, exe_or_dll):
    if exe_or_dll in dll_dict and dll_dict[exe_or_dll]:
        return dll_dict
    dll_dict[exe_or_dll] = True
    # print(f"dumpbin /IMPORTS {exe_or_dll}")
    command = [DUMPBIN, '/IMPORTS', exe_or_dll]
    result = subprocess.run(command, stdout=subprocess.PIPE)
    for line in result.stdout.decode().split('\r\n'):
        if DLL_RE.match(line):
            dll = line.strip()
            dll = f"{VTK_BIN}\\{dll}"
            if dll not in dll_dict:
                dll_dict[dll] = False
    for x in dll_dict:
        if not dll_dict[x]:
            return depends(dll_dict, x)
    return dll_dict

dlls = depends(dict(), MV_DLL)
dlls = depends(dlls, MVMF6_EXE)

dlls.pop(MV_DLL)
dlls.pop(MVMF6_EXE)

split_re = re.compile('.*vtk(.*)-9.1.dll')

print('# check for vtk')
print('find_package(VTK REQUIRED)')
print('if (${VTK_VERSION} VERSION_EQUAL "9.1")')
print("  find_package(VTK")
print("    COMPONENTS")
for k in sorted(dlls.keys(), key=str.lower):
    m = split_re.match(k)
    print(f"      {m.group(1)}")
print("    REQUIRED")
print("  )")
print("endif()")

# check for vtk
find_package(VTK REQUIRED)
if (${VTK_VERSION} VERSION_EQUAL "9.1")
  find_package(VTK
    COMPONENTS
      CommonColor
      CommonComputationalGeometry
      CommonCore
      CommonDataModel
      CommonExecutionModel
      CommonMath
      CommonMisc
      CommonSystem
      CommonTransforms
      DICOMParser
      doubleconversion
      expat
      FiltersCore
      FiltersExtraction
      FiltersGeneral
      FiltersGeometry
      FiltersHybrid
      FiltersModeling
      FiltersSources
      FiltersStatistics
      fmt
      freetype
      glew
      GUISupportMFC
      ImagingCore
      ImagingSources
      InteractionStyle
      IOCore
      IOImage
      IOLegacy
      IOXML
      IOXMLParser
      jpeg
      kissfft
      loguru
      lz4
      lzma
      metaio
      ParallelCore
      ParallelDIY
      png
      pugixml
      RenderingCore
      RenderingFreeType
      RenderingLOD
      RenderingOpenGL2
      RenderingUI
      sys
      tiff
      zlib
    R